In [1]:
import pandas as pd 
import os
import numpy as np

In [2]:
def sort_by_year(file_name):

    year_part = file_name.split("_")[1]

    return year_part

In [5]:
var_to_interpolate = 'B01001_001E'
census_files = ['../assets/census/customVariables/'+i \
                for i in os.listdir('../assets/census/customVariables/')]
census_files


['../assets/census/customVariables/DEC_2000_B01001C.csv',
 '../assets/census/customVariables/DEC_2000_B01001D.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001_002E.csv',
 '../assets/census/customVariables/ACS5Y_2021_B01001_001E.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001F.csv',
 '../assets/census/customVariables/ACS5Y_2011_B01001D.csv',
 '../assets/census/customVariables/DEC_2000_B01001E.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001_001E.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001I.csv',
 '../assets/census/customVariables/ACS5Y_2011_B01001_026E.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001D.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001G.csv',
 '../assets/census/customVariables/ACS5Y_2021_B01001A.csv',
 '../assets/census/customVariables/old vars',
 '../assets/census/customVariables/ACS5Y_2021_B01001F.csv',
 '../assets/census/customVariables/ACS5Y_2021_B01001_002E.csv',
 '../assets/census/customVariables/ACS5Y

In [6]:
census_files_to_interp = sorted([i for i in census_files if var_to_interpolate in i ], key=sort_by_year)
census_files_to_interp

['../assets/census/customVariables/DEC_2000_B01001_001E.csv',
 '../assets/census/customVariables/ACS5Y_2011_B01001_001E.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001_001E.csv',
 '../assets/census/customVariables/ACS5Y_2021_B01001_001E.csv']

In [7]:
df = pd.read_csv(census_files_to_interp[0])
df

,PAT_ZIP,B01001_001E
0,73949,1866
1,75001,8175
2,75002,37171
3,75006,45990
4,75007,50357
...,...,...
1831,79936,92136
1832,79938,18379
1833,88063,10274
1834,88220,33185


In [119]:
df.columns[2]

'B01001A_002E'

In [75]:
def div_by_population(column, total_pop):
    return column / df[total_pop]

In [138]:
def processPopulationByAge(base_df):
    df = base_df.drop('PAT_ZIP',axis=1)

    male_df = df.drop(base_df.columns[2], axis=1).iloc[:,1:15].apply(lambda x: div_by_population(x, base_df.columns[2]))


    female_df = df.drop(base_df.columns[17], axis=1).iloc[:,16:].apply(lambda x: div_by_population(x, base_df.columns[17]))

    derived_df = pd.concat([male_df, female_df], axis=1).sort_index(axis=1)
    derived_df.insert(0, 'PAT_ZIP', base_df['PAT_ZIP'])

    return derived_df
    
    

    

In [137]:
processPopulationByAge(df)

,PAT_ZIP,B01001A_003E,B01001A_004E,B01001A_005E,B01001A_006E,B01001A_007E,B01001A_008E,B01001A_009E,B01001A_010E,B01001A_011E,...,B01001A_022E,B01001A_023E,B01001A_024E,B01001A_025E,B01001A_026E,B01001A_027E,B01001A_028E,B01001A_029E,B01001A_030E,B01001A_031E
0,73949,0.081807,0.085470,0.105006,0.050061,0.040293,0.029304,0.057387,0.056166,0.152625,...,0.014686,0.064085,0.052069,0.082777,0.166889,0.114820,0.116155,0.065421,0.069426,0.028037
1,75001,0.033171,0.020390,0.017651,0.005782,0.009130,0.079428,0.196896,0.167985,0.186853,...,0.019164,0.076657,0.162425,0.127867,0.192900,0.146717,0.081370,0.033616,0.027647,0.002828
2,75002,0.092547,0.100305,0.096701,0.051863,0.026145,0.028283,0.064692,0.089493,0.226695,...,0.018126,0.032688,0.071275,0.091367,0.228018,0.130323,0.060031,0.024516,0.007803,0.002642
3,75006,0.077040,0.067749,0.062251,0.038994,0.025722,0.066865,0.083612,0.087341,0.179296,...,0.022335,0.062315,0.071878,0.080577,0.175037,0.140116,0.095077,0.048433,0.015795,0.012957
4,75007,0.073369,0.080859,0.088787,0.044831,0.021759,0.046744,0.064185,0.072276,0.225466,...,0.014760,0.037878,0.064223,0.084960,0.218166,0.166958,0.069883,0.026821,0.008729,0.004814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,79936,0.097334,0.114269,0.100923,0.059614,0.025908,0.061246,0.065194,0.077952,0.176200,...,0.028144,0.056347,0.075080,0.081244,0.162367,0.118347,0.045342,0.026881,0.010794,0.006164
1832,79938,0.073195,0.096640,0.073910,0.051322,0.064475,0.163545,0.078056,0.076197,0.165547,...,0.034588,0.069941,0.069176,0.087139,0.221288,0.104720,0.070132,0.024651,0.007262,0.006879
1833,88063,0.089720,0.123783,0.114659,0.067214,0.048358,0.086983,0.052920,0.044100,0.126217,...,0.038873,0.079311,0.059223,0.050613,0.108270,0.082703,0.046961,0.060005,0.016697,0.000000
1834,88220,0.065102,0.077915,0.079196,0.044843,0.031150,0.046365,0.051890,0.047886,0.156230,...,0.017956,0.051060,0.052464,0.050913,0.144166,0.135299,0.093401,0.068425,0.054903,0.025715


In [88]:
def processEducationData(base_df, male=True):
    df = base_df.drop('PAT_ZIP',axis=1)

    if male:
        total_pop_column = 'B15002_002E'
    else:
        total_pop_column = 'B15002_019E'

    derived_df = df.drop(total_pop_column, axis=1).apply(lambda x: div_by_population(x, total_pop_column))
    derived_df.insert(0, 'PAT_ZIP', base_df['PAT_ZIP'])

    return derived_df

In [81]:
def processPovertyData(base_df):
    df = base_df.drop('PAT_ZIP',axis=1)
    derived_df = pd.DataFrame()

    var_names = sorted(list(set([i.split('_')[0] for i in df.columns if 'PAT_ZIP' not in i])))

    for ind, key in enumerate(var_names):
        # print(key)
        temp_df = df.filter(like=key, axis=1)
        # print(temp_df.filter(like='002',axis=1))
        divided = temp_df.filter(like='002', axis=1).values/temp_df.filter(like='001', axis=1).values
        divided = divided.ravel()
        # print(divided.shape)
        derived_df.insert(ind, key, divided)
        # print(temp_df.columns)
    derived_df.insert(0, 'PAT_ZIP', base_df['PAT_ZIP'])
    return derived_df


In [82]:
census_files_to_interp

['../assets/census/customVariables/DEC_2000_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2011_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2016_B15002M.csv',
 '../assets/census/customVariables/ACS5Y_2021_B15002M.csv']

In [140]:
census_files_to_interp

['../assets/census/customVariables/DEC_2000_B01001A.csv',
 '../assets/census/customVariables/ACS5Y_2011_B01001A.csv',
 '../assets/census/customVariables/ACS5Y_2016_B01001A.csv',
 '../assets/census/customVariables/ACS5Y_2021_B01001A.csv']

In [143]:
letters_list = ['A','B','C','D','E','F','G','H','I']
var_list = ['B01001'+i for i in letters_list]
var_list

['B01001A',
 'B01001B',
 'B01001C',
 'B01001D',
 'B01001E',
 'B01001F',
 'B01001G',
 'B01001H',
 'B01001I']

In [145]:
letters_list = ['A','B','C','D','E','F','G','H','I']
var_list = ['B01001'+i for i in letters_list]
for variable in var_list:
    var_to_interpolate = variable
    census_files_to_interp = sorted([i for i in census_files if var_to_interpolate in i ], key=sort_by_year)
    for file in census_files_to_interp:
        df = pd.read_csv(file)
        if 'state' in df.columns:
            df = df.drop('state', axis=1)
        
        # cleaned_df = processPovertyData(df)
        # cleaned_df = processEducationData(df, male=False)
        cleaned_df = processPopulationByAge(df)

        # cleaned_df = df.drop(['B02001_001E','PAT_ZIP'],axis=1).apply(divide_by_total)
        # don't need to normalize for B19013
        # cleaned_df = df.drop(['PAT_ZIP'],axis=1)
        # cleaned_df.insert(0,'PAT_ZIP',df['PAT_ZIP'])
        # cleaned_df = cleaned_df.dropna()
        cleaned_df.to_csv(f'../assets/census/derivedVariables/{sort_by_year(file)}_{var_to_interpolate}.csv',index=False)
        
    

In [142]:

for file in census_files_to_interp:
    df = pd.read_csv(file)
    if 'state' in df.columns:
        df = df.drop('state', axis=1)
    
    # cleaned_df = processPovertyData(df)
    # cleaned_df = processEducationData(df, male=False)
    cleaned_df = processPopulationByAge(df)

    # cleaned_df = df.drop(['B02001_001E','PAT_ZIP'],axis=1).apply(divide_by_total)
    # don't need to normalize for B19013
    # cleaned_df = df.drop(['PAT_ZIP'],axis=1)
    # cleaned_df.insert(0,'PAT_ZIP',df['PAT_ZIP'])
    # cleaned_df = cleaned_df.dropna()
    cleaned_df.to_csv(f'../assets/census/derivedVariables/{sort_by_year(file)}_{var_to_interpolate}.csv',index=False)
    
    

In [95]:
file_path = "../assets/census/derivedVariables/"
B15_files = [file_path+i for i in os.listdir(file_path) if '.csv' in i]

In [107]:
B15_files

['../assets/census/derivedVariables/2000_B15002F.csv',
 '../assets/census/derivedVariables/2000_B15002M.csv',
 '../assets/census/derivedVariables/2011_B15002F.csv',
 '../assets/census/derivedVariables/2011_B15002M.csv',
 '../assets/census/derivedVariables/2016_B15002F.csv',
 '../assets/census/derivedVariables/2016_B15002M.csv',
 '../assets/census/derivedVariables/2021_B15002F.csv',
 '../assets/census/derivedVariables/2021_B15002M.csv']

In [115]:
for i in range(4):
    # print(i*2, i*2+2)

    df1_path, df2_path = B15_files[i*2:i*2+2]
    df1 = pd.read_csv(df1_path)
    df2 = pd.read_csv(df2_path)

    df = df1.merge(df2, on='PAT_ZIP')
    df.to_csv(f'{df1_path[:-4]}.csv',index=False)

In [63]:
df = pd.read_csv(census_files_to_interp[0])
df

,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E,state,PAT_ZIP
0,12510,9170,1444,34,686,136,891,149,48,75001
1,62175,46812,6156,90,5272,0,1470,2375,48,75002
2,45931,35798,3461,328,3206,0,2525,613,48,75006
3,50921,35589,4542,593,6820,30,1634,1713,48,75007
4,8418,7387,112,207,0,0,505,207,48,75009
...,...,...,...,...,...,...,...,...,...,...
1930,17995,12349,2694,9,332,16,1481,1114,48,79934
1931,18249,14543,584,89,98,22,2527,386,48,79935
1932,114941,96682,2237,576,737,0,12809,1900,48,79936
1933,51367,42417,1381,437,401,72,5320,1339,48,79938


In [7]:
def divide_by_total(column, population_column):
    return column/df['B02001_001E']

In [74]:
cleaned_df = df.drop(['B02001_001E','state','PAT_ZIP'],axis=1).apply(divide_by_total)
cleaned_df.insert(0,'PAT_ZIP',df['PAT_ZIP'])
cleaned_df

,PAT_ZIP,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,B02001_008E
0,75001,0.733014,0.115428,0.002718,0.054836,0.010871,0.071223,0.011910
1,75002,0.752907,0.099011,0.001448,0.084793,0.000000,0.023643,0.038199
2,75006,0.779386,0.075352,0.007141,0.069800,0.000000,0.054974,0.013346
3,75007,0.698906,0.089197,0.011645,0.133933,0.000589,0.032089,0.033640
4,75009,0.877524,0.013305,0.024590,0.000000,0.000000,0.059990,0.024590
...,...,...,...,...,...,...,...,...
1930,79934,0.686246,0.149708,0.000500,0.018450,0.000889,0.082301,0.061906
1931,79935,0.796920,0.032002,0.004877,0.005370,0.001206,0.138473,0.021152
1932,79936,0.841145,0.019462,0.005011,0.006412,0.000000,0.111440,0.016530
1933,79938,0.825764,0.026885,0.008507,0.007807,0.001402,0.103568,0.026067


In [75]:
cleaned_df.to_csv('../assets/census/derivedVariables/2011_B02001.csv',index=False)

In [48]:
import cenpy
dataset = 'ACSDT5Y2011'
conn = cenpy.remote.APIConnection(dataset)

In [49]:
conn_df = conn.variables

In [68]:
for i in range(0,8):
      print(conn_df.loc[df.columns[i]]['label'], '\n', 
            conn_df.loc[df.columns[i]]['concept'])

Estimate!!Total 
 RACE
Estimate!!Total!!White alone 
 RACE
Estimate!!Total!!Black or African American alone 
 RACE
Estimate!!Total!!American Indian and Alaska Native alone 
 RACE
Estimate!!Total!!Asian alone 
 RACE
Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone 
 RACE
Estimate!!Total!!Some other race alone 
 RACE
Estimate!!Total!!Two or more races 
 RACE
